## Data Maker
plays random games labels them then saves the data so a model can be trained on it later. The idea was to save time on the first 10000 or games by saving fully random games then training. Does not implement piece values correctly now and models didnt train well on this data.

In [29]:
import chess
import numpy as np
import random
from IPython.display import clear_output
from chessbot.chessbot import ChessBot
from chessbot.model_input import ModelInput

def move(board):
        moves = list(board.legal_moves)
        move = random.choice(moves)
        
        return move


def get_piece_values(board):
	reward = 0
	piece_map = board.piece_map()
	for _, piece in piece_map.items():
		color = 1 if piece.color else -1


		if piece.piece_type == chess.ROOK:
			reward += 5*color
		elif piece.piece_type == chess.BISHOP or piece.piece_type == chess.KNIGHT:
			reward += 3*color
		elif piece.piece_type == chess.QUEEN:
			reward += 8*color
		elif piece.piece_type == chess.PAWN:
			reward += 1*color
		r = reward/38 if (abs(reward/38))<1 else (reward/abs(reward))
	return r

def play_game():
	moves = []
	reward = []
	board = chess.Board()
	while not board.is_game_over(claim_draw=True):
		m = move(board)
		board.push(m)
		moves.append(ModelInput(board).get_input())
		reward.append(get_piece_values(board))

	result = board.outcome(claim_draw=True).result()


	# Blend the moves into a single array with alternating elements
	X = np.array(moves)
	y = np.array(reward)
	# Set the label for the last move to 1, representing a winning move, then
	# discount the rest as they led to a win but should not be rewarded as heavily
	discount_factor = 0.98
	if result != '1/2-1/2':
		# If black won flip the labels since our model evaluates white's position
		for i in range(5,0,-1):
			if result == '1-0':
				y[len(y)-i] = 1*discount_factor**(i-1)
			elif result == '0-1':
				y[len(y)-i] = -1*discount_factor**(i-1)



	# Scale the labels to be between 0 and 1 instead of -1 and 1
	y = (y + 1) / 2

	return result, X, y

# TODO

Fix so it uses the piece values labeling scheme and alter to save both wins and draws

In [33]:
from IPython.display import clear_output

results = []
white_wins = 0
black_wins = 0
draws = 0

X_all = None
y_all = None

while (white_wins+black_wins)<=10000:
	print(f'\rWins: {white_wins + black_wins}/10000 Total games: {white_wins+black_wins+draws}', end='')
	result, X, y = play_game()
	results.append(result)

	if X is not None and y is not None:
		if X_all is None:
			X_all = X
			y_all = y
		else:
			X_all = np.concatenate((X_all, X))
			y_all = np.concatenate((y_all, y))

	if result == '1-0':
		white_wins += 1
	if result == '0-1':
		black_wins += 1
	if result == '1/2-1/2':
		draws += 1
	

# Save the training data 
np.savez_compressed(f'chessbot_wins_data/{len(X_all)}_moves.npz', X=X_all, y=y_all)

Wins: 10000/10000 Total games: 66245